In [2]:
import csv
import argparse

import numpy as np

from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ExpSineSquared

In [45]:
class PFSP:
    def __init__(self):
        self.p = None
        self.n_jobs = None
        self.m_machines = None

    def C_init(self, sigma):
        c = np.zeros(self.p.shape)
        c[0, 0] = self.p[sigma[0], 0]
        for i in range(self.n_jobs):
            c[i, 0] = self.p[sigma[i], 0] + c[i - 1, 0]
        for j in range(self.m_machines):
            c[0, j] = self.p[sigma[0], j] + c[0, j - 1]
        for i in range(1, self.n_jobs):
            for j in range(1, self.m_machines):
                c[i, j] = self.p[sigma[i], j] + max(c[i - 1, j], c[i, j - 1])
        return c

    def F(self, permutation):
        summation = 0
        c = self.C_init(permutation)
        for i in range(len(permutation)):
            summation -= c[i, self.m_machines - 1]
        return summation

    def random_key(self, v):
        permutation = np.argsort(v)
        return permutation

    def black_box_function(self, **kwargs):
        data = np.fromiter(kwargs.values(), dtype=float)
        permutation = self.random_key(data)
        cost = self.F(permutation)
        return cost

    def read_fsp_file(self, filename):
        with open(filename) as f:
            content = f.readlines()

        # Extract number of jobs and machines from the second line
        header = list(map(int, ' '.join(content[1].split()).split()))
        n_jobs, m_machines = header[0], header[1]

        # Extract the processing times of jobs
        p_times_str = content[3:3 + int(m_machines)]
        p_matrix = []
        for row in p_times_str:
            p_matrix.append(list(map(int, ' '.join(row.split()).split())))
        p_times = np.matrix(p_matrix).transpose()

        return n_jobs, m_machines, p_times

    def generate_list_of_files_taillard(self, jobs=(20, 50, 100, 200, 500), machines=(5, 10, 20),
                                        instances=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)):
        list_of_files = []
        for job in jobs:
            for machine in machines:
                for instance in instances:
                    file = "taillard_benchmark/tai{}_{}_{}.fsp".format(job, machine, instance)
                    list_of_files.append(file)
        return list_of_files

    def generate_list_of_files_random(self, jobs=(250, 300, 350, 400, 450), machines=(10, 20),
                                      instances=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)):
        list_of_files = []
        for job in jobs:
            for machine in machines:
                for instance in instances:
                    file = "random_benchmark/cebe{}_{}_{}.fsp".format(job, machine, instance)
                    list_of_files.append(file)
        return list_of_files

    def generate_bounds(self, n, lower_bound=0, upper_bound=1):
        i = 0
        pbounds = {}
        while i < n:
            xi = 'x' + str(i)
            pbounds[xi] = (lower_bound, upper_bound)
            i += 1
        return pbounds

    def execute_kalimero(self, file, seed, it=1000, nu=2.5, kappa=2.5, xi=0.0):
        self.n_jobs, self.m_machines, self.p = self.read_fsp_file(file)
        target_results = []
        points_results = []

        # Bounds of each variable
        pbounds = self.generate_bounds(self.n_jobs)

        # Bayesian Optimizer
        optimizer = BayesianOptimization(
            f=None,
            pbounds=pbounds,
            verbose=2,
            random_state=seed,
        )

        # Kernel
        optimizer.set_gp_params(kernel=Matern(nu=nu))

        # Adquisition function
        utility = UtilityFunction(kind="ucb", kappa=kappa, xi=xi)

        # Bayesian Optimization with Gaussian Process
        for _ in range(it):
            next_point = optimizer.suggest(utility)
            target = self.black_box_function(**next_point)
            optimizer.register(params=next_point, target=target)
            points_results.append(next_point)
            target_results.append(int(target))

        savefile = 'results/' + file[21:-4]+'__{}_{}_{}'.format(nu,kappa,seed)
        row = (file, int(optimizer.max['target']), seed, nu, kappa)
        np.save(savefile, row)

    def execute(self, file, seed, it=1000, nu=2.5, kappa=2.5, xi=0.0):
        self.n_jobs, self.m_machines, self.p = self.read_fsp_file(file)
        
        # Bounds of each variable
        pbounds = self.generate_bounds(self.n_jobs)

        # Bayesian Optimizer
        optimizer = BayesianOptimization(
            f=None,
            pbounds=pbounds,
            verbose=2,
            random_state=seed,
        )



        optimizer.maximize(
            init_points=2,
            n_iter=it,
        )


        return res

    def execute_taillard_benchmarks(self, jobs=(20, 50, 100, 200, 500), machines=(5, 10, 20),
                                    instances=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), seed=0, it=1000):
        with open("taillard_results.csv", "w") as fp:
            writer = csv.writer(fp, delimiter=",", lineterminator="\n")
            # writer.writerow(["your", "header", "foo"])  # write header
            writer.writerow(['file', 'points_results', 'target_results', 'best_result', 'seed', 'n_it'])
        list_of_files = self.generate_list_of_files_taillard(jobs, machines, instances)
        for file in list_of_files:
            file, points_results, target_results, best_result, seed, it = self.execute(file, seed, it)
            # Write CSV file
            with open("taillard_results.csv", "a") as fp:
                writer = csv.writer(fp, delimiter=",", lineterminator="\n")
                writer.writerow((file, int(best_result['target']), seed, it))

    def execute_random_benchmarks(self, jobs=(250, 300, 350, 400, 450), machines=(10, 20),
                                  instances=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), seed=0, it=1000):
        with open("random_results.csv", "w") as fp:
            writer = csv.writer(fp, delimiter=",", lineterminator="\n")
            # writer.writerow(["your", "header", "foo"])  # write header
            writer.writerow(['file', 'target_results', 'best_result', 'seed', 'n_it'])
        list_of_files = self.generate_list_of_files_random(jobs, machines, instances)
        for file in list_of_files:
            file, points_results, target_results, best_result, seed, it = self.execute(file, seed, it)
            # Write CSV file
            with open("random_results.csv", "a") as fp:
                writer = csv.writer(fp, delimiter=",", lineterminator="\n")
                writer.writerow((file, int(best_result['target']), seed, it))

In [49]:
file = 'taillard_benchmark/tai20_5_0.fsp'
seed = 0
nu = 0.1
kappa = 0.01
xi = 0.0

it = 10


In [50]:
pfsp = PFSP()

In [51]:
a = pfsp.execute_kalimero(file=file, it=it, seed = seed, nu=nu, kappa=kappa, xi=xi)